# Plagiarism Detection Notebook
## Notebook for the "Textmining" project in WS2020/2021

Sources used for code: 

* https://radimrehurek.com/gensim/auto_examples/core/run_core_concepts.html

* https://radimrehurek.com/gensim/auto_examples/core/run_corpora_and_vector_spaces.html#corpus-streaming-tutorial

* https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#12buildingthetopicmodel

In [10]:
# imports
import pprint
import gensim
from gensim import corpora
from gensim import models
from gensim import similarities
import nltk
from nltk.corpus import stopwords
from smart_open import open 



---

# First experiments

In [3]:
# some example documents. For the actual application we wouldn't load everything at once.

documents = [
    "Human machine interface for lab abc computer applications",
    "A survey of user opinion of computer system response time",
    "The EPS user interface management system",
    "System and human system engineering testing of EPS",
    "Relation of user perceived response time to error measurement",
    "The generation of random binary unordered trees",
    "The intersection graph of paths in trees",
    "Graph minors IV Widths of trees and well quasi ordering",
    "Graph minors A survey",
]

## Preprocessing

Prepocess the text

In [1]:
# simple function for text preprocessing. Converts the text to lower case, removes stopwords and words with a minimum length of 2 and maximum length of 15
def preprocessing (corpus):
    
    processed_corpus = []
    
    # load stopwords from NLTK
    stop_words = set(stopwords.words('english'))
    
    # go through each document in the corpus
    for document in corpus:
        
        # step1: convert to lowercase and remove words that do not match the min-max-length
        step1 = gensim.utils.simple_preprocess(document, deacc=False, min_len=2, max_len=15)
        
        #step2: remove stopwords
        step2 = [word for word in step1 if word not in stop_words]
        
        processed_corpus.append(step2)
    return processed_corpus


In [5]:
processed_corpus = preprocessing(documents)
pprint.pprint(processed_corpus)

[['human', 'machine', 'interface', 'lab', 'abc', 'computer', 'applications'],
 ['survey', 'user', 'opinion', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'management', 'system'],
 ['system', 'human', 'system', 'engineering', 'testing', 'eps'],
 ['relation', 'user', 'perceived', 'response', 'time', 'error', 'measurement'],
 ['generation', 'random', 'binary', 'unordered', 'trees'],
 ['intersection', 'graph', 'paths', 'trees'],
 ['graph', 'minors', 'iv', 'widths', 'trees', 'well', 'quasi', 'ordering'],
 ['graph', 'minors', 'survey']]


Maybe add a filter for min occurence?

In [8]:
# convert text to vectors using the dictionary function

# define dictionary of our corpus. Contains the word frequency of each token in the whole corpus
dictionary = corpora.Dictionary(processed_corpus)

# transform the corpus to vectors. Each vector consists of a token ID and the token frequency (taken from the dictionary)
bow_corpus = [dictionary.doc2bow(text) for text in processed_corpus]

print(dictionary)
print(dictionary.token2id)
pprint.pprint(bow_corpus)


Dictionary(35 unique tokens: ['abc', 'applications', 'computer', 'human', 'interface']...)
{'abc': 0, 'applications': 1, 'computer': 2, 'human': 3, 'interface': 4, 'lab': 5, 'machine': 6, 'opinion': 7, 'response': 8, 'survey': 9, 'system': 10, 'time': 11, 'user': 12, 'eps': 13, 'management': 14, 'engineering': 15, 'testing': 16, 'error': 17, 'measurement': 18, 'perceived': 19, 'relation': 20, 'binary': 21, 'generation': 22, 'random': 23, 'trees': 24, 'unordered': 25, 'graph': 26, 'intersection': 27, 'paths': 28, 'iv': 29, 'minors': 30, 'ordering': 31, 'quasi': 32, 'well': 33, 'widths': 34}
[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1)],
 [(2, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1)],
 [(4, 1), (10, 1), (12, 1), (13, 1), (14, 1)],
 [(3, 1), (10, 2), (13, 1), (15, 1), (16, 1)],
 [(8, 1), (11, 1), (12, 1), (17, 1), (18, 1), (19, 1), (20, 1)],
 [(21, 1), (22, 1), (23, 1), (24, 1), (25, 1)],
 [(24, 1), (26, 1), (27, 1), (28, 1)],
 [(24, 1), (26, 1), (29, 1), (30, 1)

Create a TF-IDF model of the corpus:

In [72]:
# train the model
tfidf = models.TfidfModel(bow_corpus)

# transform the corpus
corpus_tfidf = tfidf[bow_corpus]


In [75]:
# every word is now represented by a vector: Token-ID and token-weight
for doc in corpus_tfidf:
    pprint.pprint(doc)

[(0, 0.4301019571350565),
 (1, 0.4301019571350565),
 (2, 0.2944198962221451),
 (3, 0.2944198962221451),
 (4, 0.2944198962221451),
 (5, 0.4301019571350565),
 (6, 0.4301019571350565)]
[(2, 0.3726494271826947),
 (7, 0.5443832091958983),
 (8, 0.3726494271826947),
 (9, 0.3726494271826947),
 (10, 0.27219160459794917),
 (11, 0.3726494271826947),
 (12, 0.27219160459794917)]
[(4, 0.438482464916089),
 (10, 0.32027755044706185),
 (12, 0.32027755044706185),
 (13, 0.438482464916089),
 (14, 0.6405551008941237)]
[(3, 0.3449874408519962),
 (10, 0.5039733231394895),
 (13, 0.3449874408519962),
 (15, 0.5039733231394895),
 (16, 0.5039733231394895)]
[(8, 0.30055933182961736),
 (11, 0.30055933182961736),
 (12, 0.21953536176370683),
 (17, 0.43907072352741366),
 (18, 0.43907072352741366),
 (19, 0.43907072352741366),
 (20, 0.43907072352741366)]
[(21, 0.48507125007266594),
 (22, 0.48507125007266594),
 (23, 0.48507125007266594),
 (24, 0.24253562503633297),
 (25, 0.48507125007266594)]
[(24, 0.31622776601683794),


## Compare Texts

In [76]:
# build an index for the corpus
index = similarities.SparseMatrixSimilarity(tfidf[bow_corpus], num_features=len(dictionary))

In [79]:
# add a query document
query_document = 'system engineering'.split()

# transform the query document to a vector
query_bow = dictionary.doc2bow(query_document)

# compare the query document to each document in the corpus
sims = index[tfidf[query_bow]]
print(list(enumerate(sims)))

[(0, 0.0), (1, 0.12172779), (2, 0.14323246), (3, 0.67615116), (4, 0.0), (5, 0.0), (6, 0.0), (7, 0.0), (8, 0.0)]


In [78]:
for document_number, score in sorted(enumerate(sims), key=lambda x: x[1], reverse=True):
    print(document_number, score)

3 0.67615116
2 0.14323246
1 0.12172779
0 0.0
4 0.0
5 0.0
6 0.0
7 0.0
8 0.0


---

# Corpus Streaming

Since we do not want to load the whole corpus into memory every time, we need some changes in our approach, so we are able to stream the single documents when needed.
Code is from: https://radimrehurek.com/gensim/auto_examples/core/run_corpora_and_vector_spaces.html#corpus-streaming-tutorial

## Preprocessing

In [11]:
# load stopwords from NLTK
stop_words = set(stopwords.words('german'))

# load all tokens as lowercase text
dictionary = corpora.Dictionary(line.lower().split() for line in open('data/corpus/deu_mixed-typical_2011_10K/deu_mixed-typical_2011_10K-sentences.txt'))

# find all stop words
stop_ids = [dictionary.token2id[stopword] for stopword in stop_words if stopword in dictionary.token2id]

# find all words that only occur once
once_ids = [tokenid for tokenid, docfreq in dictionary.dfs.items() if docfreq == 1]

# Remove all stopwords and words that only occur once
dictionary.filter_tokens(stop_ids + once_ids)

# Since we filtered out some words, the ID count now has gaps. With .compactify we can remove those gaps
dictionary.compactify()


In [6]:
# Define the corpus as an object

class MyCorpus:
    def __iter__(self):
        # Each line in the corpus file represents one document, each token in a document is seperated by a whitespace
        for line in open('data/corpus/deu_mixed-typical_2011_10K/deu_mixed-typical_2011_10K-sentences.txt'):
            # Transfom the corpus to vectors
            yield dictionary.doc2bow(line.lower().split())

In [7]:
# Initialize the corpus, without loading it into memory
corpus = MyCorpus()


Note: The corpus now exists only as an object, to work with it, the vectors inside have to be called. Only then they will be loaded into memory.
Example: Print the first 10 document vectors:

In [15]:
for index, vector in enumerate(corpus):  # load one vector into memory at a time
    if index <10:
        print(vector)

[(0, 1), (1, 1), (2, 1), (3, 1)]
[(4, 1), (5, 1), (6, 1), (7, 1)]
[(7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1)]
[(13, 1), (14, 1), (15, 1), (16, 1)]
[(7, 1), (17, 1), (18, 1), (19, 1)]
[(7, 1), (20, 1), (21, 1)]
[(7, 1), (22, 1), (23, 1), (24, 1), (25, 1)]
[(26, 1), (27, 1), (28, 1), (29, 1)]
[(30, 1), (31, 1), (32, 1), (33, 1)]
[(7, 1), (19, 1), (34, 1), (35, 1), (36, 1)]


## Work with our Corpus

Now that we are able to load our corpus memory friendly we can transform the document vectors using a variety of functions.

First we have to initialize a model, which will be used for the transformations:

In [18]:
# TF-IDF Model:
tf_idf_model = models.TfidfModel(corpus, normalize=False)
